# Set up the environment

Before running the notebook, please install the necessary packages and environment by running the following shell commands in your terminal:

```bash
# Create the conda environment from the provided environment file
conda env create -f ../conda_env_pkgs.yml -n soc_model_env

# Activate the new environment
conda activate soc_model_env

# Launch Jupyter Notebook from within the environment
jupyter notebook


In [ ]:
import os
import sys

import nbimporter


import ee
import geemap

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#Import relevant functions from soc_predictive_model.ipynb
from soc_predictive_model.ipynb

# Authenticate and Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project= "ee-christopherharrellgis")

#Initialize Map
Map = geemap.Map()

ModuleNotFoundError: No module named 'nbimport'

### Data Imports

In [ ]:
# SOC samples table loaded from earth engine asset
soc_sample_points = ee.FeatureCollection("users/christopherharrellgis/soc_samples")

# Buffer sample points by 10km and return bounds
study_area = soc_sample_points.geometry().bounds().buffer(10000).bounds()

# Harmonized Sentinel-2 SR and Cloud Score
s2_sr = ee.ImageCollection('COPERNICUS/S2_SR')
s2CloudScore = ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED')



##### Define Global Variables

In [ ]:
# Global variables
output_data_folder = "../soc/data/"

# Start/End Dates
p1_start_date = "2022-03-01"
p1_end_date = "2023-02-28"
 
p2_start_date = "2023-03-01"
p2_end_date = "2024-02-19"

# Sentinel-2
QA_Band = "cs_cdf"
CLEAR_THRESHOLD = 0.60
S2_SCALE_FACTOR = 0.0001